In [1]:
""" Alpaca Paper Trade with the 1st LSTM """

import os

if "src" not in os.listdir():
    os.chdir("../../../")

import json

import time
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
import pickle
import warnings

import pandas_market_calendars as mcal
from keras.models import load_model

from src.lib.alpaca_historical import AlpacaData
from src.lib.alpaca_paper import AlpacaTrader
from src.lib.activations import negative_softmax
from src.lib.price_history import transform_price_history, PriceHistory
from src.lib.stock_dataset import join_price_histories
from src.lib.util import makedir_to
%load_ext autoreload
%autoreload 2

In [2]:
path = "results/4_lstm_long/"
keys = "alpaca_config.json"
isPaper = True

In [3]:
def now_utc():
    return datetime.now().astimezone(tz=timezone("UTC"))


def get_next_trading_minute():
    nyse = mcal.get_calendar("NYSE")
    now = now_utc()
    lookahead = timedelta(days=0)
    while True:
        lookahead += timedelta(hours=1)

        schedule = nyse.schedule(start_date=now, end_date=now + lookahead)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
            valid_minutes = mcal.date_range(schedule, frequency="1T")            
        
        where_future = np.where(valid_minutes > now)[0]
        if len(where_future) > 0:
            break

    return valid_minutes[where_future[0]]

# def get_last_n_minutes(n_valid):
#     nyse = mcal.get_calendar("NYSE")
#     now = now_utc()
#     lookbehind = timedelta(days=0)
#     n_iter = 0
#     while True:
#         lookbehind += timedelta(minutes=n_valid * 2**n_iter)
#         n_iter += 1

#         schedule = nyse.schedule(start_date=now - lookbehind, end_date=now)

#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             # PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
#             valid_minutes = mcal.date_range(schedule, frequency="1T")            
        
#         where_past = np.where(valid_minutes < now)[0]
#         if len(where_past) > n_valid:
#             break

#     return valid_minutes[where_past[-n_valid:]]


# def get_last_n_candles(n_candles, symbol, next_trade_time):
#     valid_mins = get_last_n_minutes(n_candles * 2)
#     data_start_time = valid_mins[0]

#     up_to_date_history = False
#     while not up_to_date_history:
#         history = api_data.get_bars(symbol, data_start_time, next_trade_time, log=False)
#         up_to_date_history = history.index[-1] == (next_trade_time - timedelta(minutes=1))
#         if not up_to_date_history:
#             time.sleep(0.5)
    
#     return history


In [4]:
api_data = AlpacaData(keys)
api_trade = AlpacaTrader(keys, isPaper)
model = load_model(
    os.path.join(path, "model.h5"),
    custom_objects={
        "negative_softmax": negative_softmax
    }
)

with open(os.path.join(path, "config.json"), "r") as f:
    config = json.load(f)

n_time_steps = config["n_time_steps"]
symbols = config["symbols"]
target_column = config["target_column"]

with open(os.path.join(path, "scaler.pkl"), "rb") as f:
    scaler = pickle.load(f)

ta_preprocess_candles = 296

running_candles = {}
for symbol in symbols:
    data_end_time = now_utc() - timedelta(minutes=10)
    running_start_time = data_end_time - timedelta(days=5)
    running_candles[symbol] = api_data.get_bars(symbol, running_start_time, data_end_time, log=False)


In [7]:

def trade_loop():

    # Sleep until next trading time
    next_trade_time = get_next_trading_minute()
    sleep_seconds = (next_trade_time - now_utc()).total_seconds() + 1.5
    print("Sleeping for", timedelta(seconds=sleep_seconds))
    time.sleep(sleep_seconds)


    # Get the recent data
    price_histories = []
    for symbol in symbols:
        old_history = running_candles[symbol]
        
        up_to_date_history = False
        while not up_to_date_history:
            recent_history = api_data.get_bars(symbol, old_history.index[-1], next_trade_time, log=False).iloc[1:]
            up_to_date_history = len(recent_history) > 0 and recent_history.index[-1] == (next_trade_time - timedelta(minutes=1))
            up_to_date_history = True
            if not up_to_date_history:
                time.sleep(0.5)
        history = old_history.append(recent_history)
        running_candles[symbol] = history
            
        if len(history) == 0:
            print(f"Warning: No data between dates")
            return
        
        price_history = PriceHistory(history, symbol)
        # TODO accept **kwargs here
        price_history = transform_price_history(price_history)
        price_histories.append(price_history)
    
    # Process the candes
    data, _, _ = join_price_histories(price_histories, target_column)
    data = scaler.transform(data)

    X = data[-n_time_steps:][np.newaxis, :, :]
    
    # Model makes predictions
    preds = model.predict(X)[0]
    next_holdings = dict(zip(symbols, preds))
    print("Desired holdings", next_holdings)
    api_trade.update_holdings(next_holdings, verbose=True)

    equity = api_trade.get_account().json()["equity"]

    now_str = str(now_utc())

    with open(os.path.join(path, "trade_log.txt"), "a") as f:
        f.write('%s,%s,%s\n' % (now_str, str(next_holdings), equity))
    
    X_save_path = os.path.join(path, "Xs", f"{now_str}.npy")
    makedir_to(X_save_path)
    np.save(X_save_path, X)

while True:
    trade_loop()

Sleeping for 0:00:47.861380
Desired holdings {'SPY': 0.1899109, 'SPXS': 0.8100249}
sell SPY 168
buy SPXS 3302
Sleeping for 0:00:58.214137
Desired holdings {'SPY': 0.8042836, 'SPXS': 0.19490527}
buy SPY 137
sell SPXS 2704
Sleeping for 0:00:58.353038
Desired holdings {'SPY': 0.13404782, 'SPXS': 0.865941}
sell SPY 149
buy SPXS 2943
Sleeping for 0:00:58.574699
Desired holdings {'SPY': 0.9839707, 'SPXS': 0.01082361}
buy SPY 189
sell SPXS 3752
Sleeping for 0:00:58.354718
Desired holdings {'SPY': 0.9158783, 'SPXS': 0.07942926}
sell SPY 14
buy SPXS 301
Sleeping for 0:00:58.551486
Desired holdings {'SPY': 0.7696833, 'SPXS': 0.22931097}
sell SPY 33
buy SPXS 658
Sleeping for 0:00:58.419801
Desired holdings {'SPY': 0.94030684, 'SPXS': 0.04991663}
buy SPY 38
sell SPXS 787
Sleeping for 0:00:58.277397
Desired holdings {'SPY': 0.8961873, 'SPXS': 0.09757344}
sell SPY 9
buy SPXS 209
Sleeping for 0:00:58.427434
Desired holdings {'SPY': 0.78564584, 'SPXS': 0.011041794}
sell SPY 25
sell SPXS 380
Sleeping f

KeyboardInterrupt: 